<a href="https://colab.research.google.com/github/Skrish0731/Brain-Tumor-Detection/blob/main/Glaucoma_Colab_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
df='/content/Images.zip'

In [ ]:
df1=zipfile.ZipFile(df,'r')

In [ ]:
df1.extractall('/content/Images')

In [ ]:
df1.close()

In [ ]:
df1 = zipfile.ZipFile(df, 'r')
df1.extractall('/content/Images')
df1.close() 

In [ ]:
os.listdir('/content/Images/Images')

['Glaucoma images', '0.0.Normal']

In [ ]:
train_normal_dir = os.path.join('/content/Images/Images/0.0.Normal')


In [ ]:
train_retinopathy_dir = os.path.join('/content/Images/Images/Glaucoma images')

In [ ]:
train_normal_names = os.listdir(train_normal_dir)
print(train_normal_names[:10])


['1ffa9633-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa962f-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa9636-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa962e-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa9644-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa9641-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa9649-8d87-11e8-9daf-6045cb817f5b..JPG', '1ffa9639-8d87-11e8-9daf-6045cb817f5b..JPG', 'Thumbs.db', '1ffa9638-8d87-11e8-9daf-6045cb817f5b..JPG']


In [ ]:
train_retinopathy_names = os.listdir(train_retinopathy_dir)
print(train_retinopathy_names[:10])


['image9.jfif', 'image30.jfif', 'image8.jfif', 'image29.jfif', 'image25.jfif', 'Thumbs.db', 'image24.jfif', 'image17.jfif', 'image20.jfif', 'image5.jfif']


In [ ]:
print('total training normal images:', len(os.listdir(train_normal_dir)))
print('total training retinopathy images:', len(os.listdir(train_retinopathy_dir)))

total training normal images: 39
total training retinopathy images: 30


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

In [ ]:
pic_index = 0

In [ ]:
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

<Figure size 1152x1152 with 0 Axes>

In [ ]:

pic_index += 8
next_normal_pix = [os.path.join(train_normal_dir, fname) 
                for fname in train_normal_names[pic_index-8:pic_index]]
next_retinopathy_pix = [os.path.join(train_retinopathy_dir, fname) 
                for fname in train_retinopathy_names[pic_index-8:pic_index]]

In [ ]:
import tensorflow as tf


In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 33, 33, 64)       

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)


In [ ]:
train_generator = train_datagen.flow_from_directory('/content/Images/Images',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 38 images belonging to 2 classes.


In [ ]:
'''his=model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=8,
      verbose=1)'''
'''
data_gen_train = ImageDataGenerator(rescale=1/255.)

data_gen_valid = ImageDataGenerator(rescale=1/255.)

train_generator = data_gen_train.flow_from_directory(train_datagen, target_size=(128,128), batch_size=128, class_mode="binary")

valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")'''

model.fit(train_generator,steps_per_epoch=1, epochs=8)



Epoch 1/8
1/1 [==============================] - 8s 8s/step - loss: 0.6786 - acc: 1.0000
Epoch 2/8
1/1 [==============================] - 7s 7s/step - loss: 0.0054 - acc: 1.0000
Epoch 3/8
1/1 [==============================] - 7s 7s/step - loss: 1.3865e-10 - acc: 1.0000
Epoch 4/8
1/1 [==============================] - 7s 7s/step - loss: 1.3865e-10 - acc: 1.0000
Epoch 5/8
1/1 [==============================] - 7s 7s/step - loss: 1.3865e-10 - acc: 1.0000
Epoch 6/8
1/1 [==============================] - 7s 7s/step - loss: 1.3865e-10 - acc: 1.0000
Epoch 7/8
1/1 [==============================] - 7s 7s/step - loss: 1.3865e-10 - acc: 1.0000
Epoch 8/8
1/1 [==============================] - 7s 7s/step - loss: 1.3865e-10 - acc: 1.0000


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is normal ")
  else:
    print(fn + " is glaucoma  ")

Saving 1ffa9652-8d87-11e8-9daf-6045cb817f5b..JPG to 1ffa9652-8d87-11e8-9daf-6045cb817f5b..JPG
[0.]
1ffa9652-8d87-11e8-9daf-6045cb817f5b..JPG is glaucoma  
